In [1]:
from tensorflow import keras 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D as Conv2D
from tensorflow.keras.layers import MaxPooling2D as MaxPooling2D
from tensorflow.keras import regularizers, optimizers
from matplotlib import pyplot as plt
import cv2
import pandas as pd
import numpy as np
import os

Using TensorFlow backend.


In [2]:
f = open('keypoints.csv')
keypoints_frame = pd.read_csv(f, delimiter = ' ', header = None , names = ['x', 'y', 'visible'])

In [3]:
keypoints_frame.head()

,x,y,visible
0,394,139,2
1,414,118,2
2,333,231,2
3,351,216,2
4,0,0,0


In [4]:
data = keypoints_frame.to_numpy()

In [5]:
aug_data = np.zeros((data.shape[0]//2, data.shape[1]*2), dtype = np.int32)
aug_data.shape

(11105, 6)

In [6]:
for i in range(0,aug_data.shape[0]):
    aug_data[i, 0:3] = data[2*i, 0:3]
    aug_data[i, 3:6] = data[2*i +1, 0:3]

In [7]:
aug_data[0, :]

array([394, 139,   2, 414, 118,   2])

In [8]:
corrected_data = pd.DataFrame({'x1':aug_data[:, 0],'y1':aug_data[:, 1],'vis1':aug_data[:, 2],'x2':aug_data[:, 3],'y2':aug_data[:, 4],'vis2':aug_data[:, 5]})

In [9]:
corrected_data.tail()

,x1,y1,vis1,x2,y2,vis2
11100,344,177,2,305,105,2
11101,358,153,2,258,126,2
11102,517,165,1,376,130,2
11103,323,209,2,231,209,2
11104,0,0,0,0,0,0


In [10]:
directory = "D:/train_images/"
path = []
for filename in os.listdir(directory):
    path.append(filename)

In [33]:
corrected_data.head()

,x1,y1,vis1,x2,y2,vis2,path
0,394,139,2,414,118,2,000000.jpg
1,333,231,2,351,216,2,000001.jpg
2,0,0,0,0,0,0,000002.jpg
3,336,130,2,291,111,2,000003.jpg
4,251,340,2,0,0,0,000004.jpg


In [40]:
corrected_data = corrected_data[corrected_data.vis1 != 0]
corrected_data = corrected_data[corrected_data.vis2 != 0]
corrected_data.head()

,x1,y1,vis1,x2,y2,vis2,path
0,394,139,2,414,118,2,000000.jpg
1,333,231,2,351,216,2,000001.jpg
3,336,130,2,291,111,2,000003.jpg
5,484,204,2,93,205,2,000005.jpg
6,396,173,2,360,176,2,000006.jpg


In [41]:
# now create the image data generator
datagen = ImageDataGenerator(rescale = 1./255)
train_generator = datagen.flow_from_dataframe(dataframe = corrected_data, directory = directory, x_col = "path", y_col = ['x1', 'y1', 'x2', 'y2'], class_mode = "raw", target_size = (480,580), batch_size = 8)

Found 9361 validated image filenames.


In [50]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), padding='same',input_shape=(480,580,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation=None))

In [51]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 480, 580, 32)      896       
_________________________________________________________________
activation_54 (Activation)   (None, 480, 580, 32)      0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 478, 578, 32)      9248      
_________________________________________________________________
activation_55 (Activation)   (None, 478, 578, 32)      0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 239, 289, 32)      0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 239, 289, 32)      0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 239, 289, 64)     

In [52]:
model.compile(optimizer='rmsprop',
              loss='mse', metrics = ['accuracy'])

In [53]:
model.fit_generator(generator=train_generator,
                    epochs=2)

W0506 23:08:33.092272 15268 data_adapter.py:1091] sample_weight modes were coerced from
  ...
    to  
  ['...']


Train for 1171 steps
Epoch 1/2
1171/1171 [==============================] - 1220s 1s/step - loss: 108672.5428 - accuracy: 0.5982
Epoch 2/2
1171/1171 [==============================] - 1051s 898ms/step - loss: 9986.4562 - accuracy: 0.6807


In [15]:
model.load_weights('model.h5')

In [54]:
json_model = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(json_model)
model.save_weights("model.h5")

In [55]:
img = cv2.imread('D:/train_images/000286.jpg')
img = np.expand_dims(img, axis=0)
# img.shape
# img = cv2.cvtCOLOR(img ,cv2.COLOR_BGR2RGB)
# plt.imshow(img)
ans = model.predict(img, batch_size = 2)

In [56]:
ans

array([[8753.373 , 5659.5894, 6320.8022, 5555.562 ]], dtype=float32)